In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder\
    .master("local[*]")\
        .appName('Test')\
            .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


24/02/19 20:45:59 WARN Utils: Your hostname, DESKTOP-SLEQT56 resolves to a loopback address: 127.0.1.1; using 172.25.13.138 instead (on interface eth0)
24/02/19 20:45:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/19 20:46:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


```
Leemos todos los archivos de una vez.
Como la estructura es anidada YEAR/MES hacemos */*/
```

In [3]:
df_green = spark.read.parquet('../data/raw/green/*/*/')
df_yellow = spark.read.parquet('../data/raw/yellow/*/*/')

In [4]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

No todas las columnas de DataSet están compartidas, buscamos las comunes.

In [5]:
set(df_green.columns) & set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

Unificamos algunas columnas

In [9]:
df_green = df_green.withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime')\
    .withColumnRenamed('lpep_dropoff_datetime','dropoff_datetime')
    
df_yellow = df_yellow.withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime')\
    .withColumnRenamed('tpep_dropoff_datetime','dropoff_datetime')

Volvemos a Ejecutar la comparacion de columnas, ahora vemos Drop_off y pickup

In [10]:
set(df_green.columns) & set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

Vemos que el orden no es el original, hacemos un artilugio

In [11]:
var_columns = df_yellow.columns
common_cols = []

for col in df_green.columns:
    if col in var_columns:
        common_cols.append(col)
        
print(common_cols)
        

['VendorID', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'payment_type', 'congestion_surcharge']


Ahora queremos seleccionar estas columnas de ambas columnas

In [12]:
from pyspark.sql import functions as F

In [13]:
df_green_select = df_green.select(common_cols)\
    .withColumn('service_type', F.lit('green'))
    
df_yellow_select = df_yellow.select(common_cols)\
    .withColumn('service_type', F.lit('yellow'))

Unimos los dos DataFrames

In [16]:
df_trips_data = df_green_select.unionAll(df_yellow_select)

In [17]:
df_trips_data.groupBy('service_type').count().show()

24/02/18 00:14:52 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 134955 ms exceeds timeout 120000 ms
24/02/18 00:14:52 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1053)
	at org.apache.spark.e

24/02/18 00:14:55 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:593)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:592)
	at org.apache.spar

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



Ahora usamos SQL para ver los datos

In [25]:
df_trips_data.createOrReplaceTempView('trips_data')

In [3]:
spark.sql("""
          SELECT service_type, count(*) FROM trips_data 
          group by service_type
          """).show()

NameError: name 'spark' is not defined

In [2]:
df_result = spark.sql("""
                      SELECT 
                      --Reveneue grouping
                      pickup_zone reveneue_zone,
                      date_trunc('month', pickup_datetime) as reveneue_month,
                      service_type,
                      -- Reveneue Calculation
                      sum(fare_amount) as reveneue_monthly_fare,
                      SUM(extra) as reveneue_monthly_extra
                      
                      from trips_data
                      GROUP BY reveneue_zone, reveneue_month, service_type
                      
                      """)

NameError: name 'spark' is not defined